In [1]:
import sys
import numpy as np
try:
    import docplex.mp

except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        get_ipython().system('pip install docplex ')
    else:
        get_ipython().system('pip install --user docplex  ')     


In [2]:
#Import de la bibliotheque docplex qui permet de realiser l'optimisation
from docplex.mp.environment import Environment
env = Environment()
env.print_information()
from docplex.mp.model import Model

model = Model("engins")

* system is: Windows 64bit
* Python version 3.7.3, located at: C:\Users\jibo6\Anaconda3\python.exe
* docplex is present, version is (2, 10, 155)
* CPLEX library is present, version is 12.9.0.0, located at: C:\Program Files\IBM\ILOG\CPLEX_Studio_Community129\cplex\python\3.7\x64_win64
* pandas is present, version is 0.24.2


In [3]:
#Fichier d'entree avec les couts et les horaires : 
import pandas as pd
Test = pd.read_csv('Test_troispe.csv', sep=';')

Kbis = Test['K'][0]
ck = Test['ck']
chk = Test['chk']
cd = Test['cd']
S = Test['S']
a = Test['a']
b = Test['b']
K = range(0,Kbis.astype(int))

Test

# Autre fichiers d'entree : tij, c1ij, Dij

,K,ck,chk,cd,S,a,b
0,2.0,1.0,1.0,1.0,0,7,25
1,NaN,1.0,1.0,NaN,0,7,25
2,NaN,NaN,NaN,NaN,0,7,25
3,NaN,NaN,NaN,NaN,0,7,25
4,NaN,NaN,NaN,NaN,0,7,25
5,NaN,NaN,NaN,NaN,0,7,25
6,NaN,NaN,NaN,NaN,0,7,25
7,NaN,NaN,NaN,NaN,0,7,25
8,NaN,NaN,NaN,NaN,0,7,25
9,NaN,NaN,NaN,NaN,0,7,25


In [2]:
import pandas as pd
#creation de l'ensemble N (ensemble des points)
#les points 0 et 1 représentent le dépôt (0 : départ et 1 : arrivée)
N=[0,1]

#Mettre la date souhaitée de la tournée
date = 1005

#Renommer si besoin, fichier qui correspondant à la liste des chantiers ainsi que leurs demandes
#S'assurer d'ajouter des lignes pour les pelles disponibles au dépôt
donnees = pd.read_excel('Donnees test.xlsx')

#Ajout du dépôt (à faire une seule fois, donc nécessaire que si vous utilisez un nouveau fichier)
#donnees = donnees.append(pd.Series(['','','00Depot','Depot', '', '',''], index=graphe.columns), ignore_index=True)
#donnees = donnees.append(pd.Series(['','','01Depot','Depot', '', '',''], index=graphe.columns), ignore_index=True)

#définition du nombre de type de pelle
donnees = donnees.assign(idpelle=(donnees['Type']).astype('category').cat.codes)
p=donnees["idpelle"].nunique()
display(p)
#on filtre les lignes et on ne garde que celles où il y a demande ou enlèvement et le dépôt
def creer_ensemble(): 
    return(donnees[ (donnees["Datedebut"]==date) | (donnees["Datefin"]==date) | (donnees["Datefin"]==0)  | (donnees["Ville"]=='Depot') ])
creer_ensemble()

#creation d'un ID par adresse / localisation
donnees = creer_ensemble().assign(idloc=(creer_ensemble()['Adresse']+creer_ensemble()['Ville']+creer_ensemble()['NParc']).astype('category').cat.codes)
donnees

3

,Type,NParc,Adresse,Ville,CP,Datedebut,Datefin,idpelle,idloc
0,NaN,A,00Depot,Depot,44119,NaN,NaN,-1,0
2,Mecalac 12,MC10,La Truliere,ST MARS DE COUTAIS,44680,1005.0,10002.0,0,3
3,Mecalac 12,MC10,Rue du Bois Fleurie,LA CHAPELLE SUR ERDRE,44240,1003.0,1005.0,0,5
7,Mecalac 13,MC12,Rue de St Servan,ST HERBLAIN,44800,1005.0,10007.0,1,4
8,Mecalac 13,MC12,Gare Sud,NANTES,44000,1008.0,1005.0,1,2
12,NaN,A,01Depot,Depot,44119,NaN,NaN,-1,1


In [5]:
#Creation de l'ensemble des points où il y a une demande
def creer_ensemble_demande(p): 
    demande = donnees[(donnees["Datedebut"]==date) & (donnees["idpelle"]==p)]
    #display(demande)
    l = demande['idloc']  
    l.name='N'+str(p)+'l'
    return(l)

#Creation de l'ensemble des points où il y a un enlèvement 
def creer_ensemble_enlever(p): 
    enlever = donnees[(donnees["Datefin"]==date) & (donnees["idpelle"]==p) ]    
    #display(enlever)
    e = enlever['idloc']
    e.name= 'N'+str(p)+'e'
    return(e)

#Creation de N l'ensemble des points 
for k in range(0,p) :
    creer_ensemble_demande(k)
    N=np.concatenate((N,creer_ensemble_demande(k)),axis=0)
    creer_ensemble_enlever(k)
    N = np.concatenate((N,creer_ensemble_enlever(k)),axis=0)
    
display(N)

#definition du nombre de points 
nbpoints = N.max()+1



array([0, 1, 3, 5, 4, 2])

In [6]:
#définition des ensembles demande et enlever 

demande = []
for k in range(0,p) :
    demande.append(creer_ensemble_demande(k))
    demande

In [7]:
enlever = []
for k in range(0,p) :
    enlever.append(creer_ensemble_enlever(k))

In [8]:
enlever

[3    5
 Name: N0e, dtype: int8, 8    2
 Name: N1e, dtype: int8, Series([], Name: N2e, dtype: int8)]

In [9]:
#Si une pelle est disponible alors il faut mettre sa date de fin à 0 dans le fichier d'entrée 'Donnees'
#si la pelle de type p est disponible au dépot, alors on ajoute 0 (dépôt) à l'ensemble disponible correspondant au type de pelle p

def creer_ensemble_disponible(p): 
    disponible = donnees[(donnees["Datefin"]==0) & (donnees["idpelle"]==p) & (donnees["idloc"]==0) ]    
    d = disponible['idloc']
    return(d)


In [10]:
disponible=[]
for k in range(0,p):
    disponible.append(creer_ensemble_disponible(k))


In [11]:
#Fichier d'entree : paramètre t(ij) temps de parcours de i à j 
#tous les engins ont la même vitesse

tij = pd.read_csv('tij.csv', sep=';',header=None)
#tij

In [12]:
#paramètre M(ij) matrice = b(i) + s(i) + t(ij) - a(j)
A = range(0,nbpoints)
Mij=np.zeros((nbpoints,nbpoints))
df = pd.DataFrame(Mij)

for i in A :
    for j in A :        
        df.iloc[i,j]=b.iloc[i]+S.iloc[i]+tij.iloc[i,j] - a.iloc[j]
#print(df)

In [13]:
#Fichier d'entree : paramètre D(ij) distance entre i et j

Dij = pd.read_csv('Dij.csv', sep=';',header=None)
#Dij

In [14]:
# Variables

#x ijk = 1 si le porte-engin k va de i à j
x = {(i,j,k) : model.binary_var(name='x_{0}_{1}_{2}'.format(i,j,k)) for i in A for j in A for k in K}

#heure de passage du porte-engin k en i
h = {(i,k): model.continuous_var(name='h_{0}_{1}'.format(i,k)) for i in A for k in K}


In [15]:
# Fonction objectif : min somme cout utilisation PE + cout trajets + cout chauffeur(pb) 
totalCost = model.sum(x[0,i,k]*ck.iloc[k] for i in A for k in K)+ model.sum(x[i,j,k]*cd[0]*Dij.iloc[((donnees.index[donnees['idloc']==i]).tolist())[0],((donnees.index[donnees['idloc']==j]).tolist())[0]] for i in A for j in A for k in K) + model.sum((h[1,k]-h[0,k] )*chk.iloc[0] for k in K) 

#totalCost = model.sum(x[0,i,k]*ck.iloc[k] for i in A for k in K)+ model.sum(x[i,j,k]*cd[0]*Dij.iloc[((donnees.index[donnees['idloc']==i]).tolist())[0],((donnees.index[donnees['idloc']==j]).tolist())[0]] for i in A for j in A for k in K) + model.sum((h[1,k]-h[0,k] )*chk.iloc[0] for k in K) 
model.minimize(totalCost)

In [16]:
# Contrainte 2: tout porte-engin qui part du dépôt revient au dépôt
for k in K:
    model.add_constraint(model.sum(x[0,i,k] for i in A ) == model.sum(x[j,1,k] for j in A ),ctname='contrainte 2')


In [17]:
# Contrainte 3: conservation des flux (interdiction du sur place) 
for k in K:
    for j in range(2,nbpoints):
        model.add_constraint(model.sum(x[i,j,k] for i in A ) == model.sum(x[j,i,k] for i in A),ctname='contrainte 3')

In [18]:
#contrainte 3 bis : on interdit les allers retours
for k in K:
    for i in A:
        for j in A:
            model.add_constraint(x[i,j,k]  + x[j,i,k] <= 1  ,ctname='contrainte 3 bis')

In [19]:
# Contrainte 4: definition des heures de passage, si on va de i à j alors h(i)<h(j)
for i in A :
    for j in A :
        for k in K:
            if(i != j):
                model.add_constraint(h[j,k] >= h[i,k] + S.iloc[i] + tij.iloc[i,j] - (1-x[i,j,k])*Mij[i,j],ctname='contrainte 4')
    

In [20]:
#Contrainte 5a: satisfaction de la demande

for p in range(0,p) :
    de = np.concatenate((disponible[p],enlever[p]),axis=0)
    de = de[np.logical_not(np.isnan(de))]
    display(enlever[p])
    l = np.array(demande[p])
    l = l[np.logical_not(np.isnan(l))]
    #display(l)

    for j in l :
        model.add_constraint( model.sum(x[int(i),int(j),k] for i in de for k in K) ==1,ctname='contrainte 5a')


3    5
Name: N0e, dtype: int8

8    2
Name: N1e, dtype: int8

Series([], Name: N2e, dtype: int8)

In [21]:
# Contrainte 6a : satisfaction des enlèvements

for p in range(0,p) :
    l0 = np.concatenate((demande[p],[1]),axis=0)
    l0 = l0[np.logical_not(np.isnan(l0))]

    e0 = np.array(enlever[p])
    e0 = e0[np.logical_not(np.isnan(e0))]

    for i in e0 :
        model.add_constraint(model.sum(x[int(i),int(j),k]  for j in l0 for k in K) ==1 ,ctname='contrainte 6a')
    

In [22]:
# Contrainte 7

for i in A :
    for k in K : 
        #On impose que h est une variable positive
        model.add_constraint(h[i,k] >= 0)
        
        #On impose que le porte-engin part du dépôt à l'ouverture
        model.add_constraint(h[0,k] == a[0])
        
        #On majore les heures d'arrivée
        model.add_constraint(h[i,k] <= b[i])  

In [23]:
#On oblige les PE à partir du dépôt, tous les PE sont utilisés
for k in K : 
    model.add_constraint(model.sum(x[0,int(j),k] for j in A) ==1) 


In [24]:
#Affichage des informations sur la résolution
model.print_information()
solution = model.solve(url=None, key=None,log_output=True)
model.report()

Model: engins
 - number of variables: 84
   - binary=72, integer=0, continuous=12
 - number of constraints: 184
   - linear=184
 - parameters: defaults
 - problem type is: MILP
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 88 rows and 20 columns.
MIP Presolve modified 20 coefficients.
Aggregator did 2 substitutions.
Reduced MIP has 94 rows, 62 columns, and 298 nonzeros.
Reduced MIP has 52 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (0.37 ticks)
Found incumbent of value 159.504833 after 0.03 sec. (0.69 ticks)
Probing time = 0.00 sec. (0.06 ticks)
Tried aggregator 1 time.
Reduced MIP has 94 rows, 62 columns, and 298 nonzeros.
Reduced MIP has 52 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.18 ticks)
Probing time = 0.00 sec. (0.06 ticks)
Clique table members: 32.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic

In [25]:
#Affichage de la valeur de la fonction objectif
result =totalCost.solution_value
result

141.35061111111122

In [26]:
print(model.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: engins

Minimize
 obj: 194445.444444443972 x_0_0_0 + 194445.444444443972 x_0_0_1
      + 24.759166666667 x_0_1_0 + 24.759166666667 x_0_1_1
      + 16.205555555556 x_0_2_0 + 16.205555555556 x_0_2_1
      + 30.855000000000 x_0_3_0 + 30.855000000000 x_0_3_1
      + 16.540000000000 x_0_4_0 + 16.540000000000 x_0_4_1
      + 45.617222222222 x_0_5_0 + 45.617222222222 x_0_5_1
      + 23.370277777778 x_1_0_0 + 23.370277777778 x_1_0_1
      + 194444.444444443972 x_1_1_0 + 194444.444444443972 x_1_1_1
      + 13.774444444444 x_1_2_0 + 13.774444444444 x_1_2_1
      + 24.676944444444 x_1_3_0 + 24.676944444444 x_1_3_1
      + 13.268888888889 x_1_4_0 + 13.268888888889 x_1_4_1
      + 39.439166666667 x_1_5_0 + 39.439166666667 x_1_5_1
      + 14.779722222222 x_2_0_0 + 14.779722222222 x_2_0_1
      + 13.356388888889 x_2_1_0 + 13.356388888889 x_2_1_1
      + 194444.444444443972 x_2_2_0 + 194444.444444443972 x_2_2_1
      + 19.8

In [27]:
#Affichage des resultats dans un fichier csv 
Matzeros=np.zeros((nbpoints,Kbis.astype(int)))
results = pd.DataFrame(Matzeros)

Matzerosx=np.zeros((nbpoints,nbpoints))

#resultx représente les matrices xij pour chaque k
for k in K :  
    resultx = pd.DataFrame(Matzerosx)
    for i in A :
        for j in A :
            resultx.iloc[i,j]=x[i,j,k].solution_value;
    display(resultx)

#'results' est un tableau représentant les heures de passage de chaque porte-engin (1 colonne = 1 PE) sur chaque chantier
for i in A :
    for k in K :        
        results.iloc[i,k]=h[i,k].solution_value;
display(results)

#Creation d'un fichier CSV qui s'enregistre automatiquement
results.to_csv('Results.csv', sep = ';')

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0


,0,1,2,3,4,5
0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0


,0,1
0,7.000000,7.000000
1,9.178417,8.963306
2,0.000000,7.637389
3,8.687028,0.000000
4,0.000000,8.417472
5,7.814861,0.000000


In [28]:
donnees

,Type,NParc,Adresse,Ville,CP,Datedebut,Datefin,idpelle,idloc
0,NaN,A,00Depot,Depot,44119,NaN,NaN,-1,0
2,Mecalac 12,MC10,La Truliere,ST MARS DE COUTAIS,44680,1005.0,10002.0,0,3
3,Mecalac 12,MC10,Rue du Bois Fleurie,LA CHAPELLE SUR ERDRE,44240,1003.0,1005.0,0,5
7,Mecalac 13,MC12,Rue de St Servan,ST HERBLAIN,44800,1005.0,10007.0,1,4
8,Mecalac 13,MC12,Gare Sud,NANTES,44000,1008.0,1005.0,1,2
12,NaN,A,01Depot,Depot,44119,NaN,NaN,-1,1


In [29]:
#création du fichier de sortie : un fichier CSV est créé pour chaque porte-engin 
#par exemple 'tournee0' représente la tournée du porte-engin 0
#Chaque fichier précise les points à visiter dans l'ordre
alltournee=[]
for k in K : 
 
    listevide= [0]
    listevidebis= [h[0,k].solution_value]
    listegrande = [listevide,listevidebis]
    
    j=0
    dernierajout=0
    #initialisation
    while (j<nbpoints and x[0,j,k].solution_value!=1 ):
            j=j+1
    if (j==nbpoints):
        display("Pas de tournee pour le porte-engin "+ str(k))
    
    while (j!= nbpoints and j!=1):
        listevide.append(j)
        listevidebis.append(h[j,k].solution_value)
        
        
        dernierajout=j
        j=0
        while (x[dernierajout,j,k].solution_value!=1 and j<nbpoints):
            j=j+1
    if j==1:
        listevide.append(j)
        listevidebis.append(h[j,k].solution_value)
        
        print("Voici la tournee pour le porte-engin "+ str(k))
        print(listegrande)
    alltournee.append(listegrande)
        
    tournee=pd.DataFrame(listegrande)
    tournee.to_csv('tournee'+str(k)+'.csv', sep=';')


Voici la tournee pour le porte-engin 0
[[0, 5, 3, 1], [7.0, 7.81486111111111, 8.687027777777772, 9.17841666666666]]
Voici la tournee pour le porte-engin 1
[[0, 2, 4, 1], [7.0, 7.637388888888889, 8.417472222222223, 8.963305555555557]]


In [30]:
#création d'un nouveau fichier d'entrée

graphe = pd.read_excel('Donnees test.xlsx')

#on supprime la ligne si on a pris une pelle du dépôt car celle-ci n'est plus disponible

p=donnees["idpelle"].nunique()
for p in range(0,p):
    for k in K : 
        for m in creer_ensemble_demande(p) : 
            if x[0,m,k].solution_value  >=1 : 
                pelle = p
                
                indexNames = donnees.index[(donnees['idpelle']==pelle) & (donnees['Ville']=='Depot').tolist()]
                #on supprime la ligne dont la pelle a été utilisée car la pelle n'est plus disponible au dépôt
                graphe.drop(indexNames , inplace=True)                    

                newgraphe.to_excel('graphe.xlsx')   
                

In [31]:
#on ajoute une ligne au fichier d'entrée si on ramène une pelle du dépôt qui rend la pelle disponible 
#la pelle est localisée au dépôt
graphe = pd.read_excel('graphe.xlsx')

for p in range(0,p):
    for k in K : 
        for m in creer_ensemble_enlever(p) : 
            if x[m,1,k].solution_value  >=1 : 
                #pelle représente l'id pelle de la pelle ramenée au dépôt
                pelle = p
                
                #on get la ligne correspondant à l'idpelle pour récupérer le nom de la pelle
                donneesbis = donnees[(donnees["idpelle"]==pelle)]
                nompelle = donneesbis[donneesbis.columns[0]].iloc[0]
                #on ajoute une ligne avec le nom de pelle et on la place au dépôt
                newgraphe = graphe.append(pd.Series([nompelle,'','00Depot','Depot', '', '',0], index=graphe.columns), ignore_index=True)
                #on réécrit le fichier graphe pour l'utiliser en fichier d'entrée par la suite  
                newgraphe.to_excel('graphe.xlsx')   


FileNotFoundError: [Errno 2] No such file or directory: 'graphe.xlsx'

In [32]:
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
import folium
import time 
#chargement données

donnees = donnees.assign(idloc=(donnees['Adresse']+donnees['Ville']).astype('category').cat.codes)
donnees = donnees.assign(idpelle=(donnees['Type']).astype('category').cat.codes)
#chargement données du jour 1005
donnees=donnees[(donnees["Datedebut"]==1005) | (donnees["Datefin"]==1005)| (donnees["Datefin"].isnull())]
donnees

,Type,NParc,Adresse,Ville,CP,Datedebut,Datefin,idpelle,idloc
0,NaN,A,00Depot,Depot,44119,NaN,NaN,-1,0
2,Mecalac 12,MC10,La Truliere,ST MARS DE COUTAIS,44680,1005.0,10002.0,0,3
3,Mecalac 12,MC10,Rue du Bois Fleurie,LA CHAPELLE SUR ERDRE,44240,1003.0,1005.0,0,5
7,Mecalac 13,MC12,Rue de St Servan,ST HERBLAIN,44800,1005.0,10007.0,1,4
8,Mecalac 13,MC12,Gare Sud,NANTES,44000,1008.0,1005.0,1,2
12,NaN,A,01Depot,Depot,44119,NaN,NaN,-1,1


In [33]:
import pandas as pd
geolocator = Nominatim(user_agent="specify_your_app_name_here")
#DEPOT: Hauraix TP, rue Descartes 44119 Treillières
depot=geolocator.geocode("rue Descartes Treillières") #localisation du dépôt
colors  = ["green","red","blue","purple","pink"] #ensemble couleurs des tournées: ex: tournée 1 en vert

display(donnees)
#display(adresse)
nantes=geolocator.geocode("Nantes")
map=folium.Map(location=(nantes.latitude,nantes.longitude), zoom_start=10)
t=8*3600
t=time.strftime('%H:%M', time.gmtime(t)) 
folium.Marker(location=(depot.latitude,depot.longitude)).add_to(map)

L=[]
for i in range(0,len(alltournee)):
    L.append(alltournee[i][0])

for i in range(0,len(L)):
    la=[]
    lo=[]
    for point in L[i]: # point= idloc

        index=donnees.index[donnees["idloc"]==point].tolist()[0] #index= indice du point dans le csv  
        a=donnees["Adresse"][index] #adresse de point 
        cp=donnees["CP"][index] #CP de point
        adresse=a+" "+str(int(cp)) #concatenation adresse et CP
        location = geolocator.geocode(adresse)
        
        if (point==0) | (point==1): #si point est le dépôt
            icone=folium.Icon(color='orange',icon='cloud') #icone pour le dépôt
            location=depot #localisation du dépôt (sans cette ligne, localisation==none pcq pas d'adresse dans le csv)
            
        else:
            icone=folium.Icon(color=colors[i],icon='info-sign') #icone pour point avec couleur spécifque à la tournée
        la.append(location.latitude) #remplissage de la liste des latitudes
        lo.append(location.longitude) #remplissage de la liste des longitudes
        
        coord=(location.latitude,location.longitude) #création des coordonnées de    
        folium.Marker(coord).add_to(map) #DISPO sur nouvelle version de Python:ajout sur la carte du point adresse
    #display(adresse)
    l = list(zip(la, lo)) #on crée une liste dans laquelle on a le COUPLE(la,lo)
    folium.PolyLine(l,color=colors[i], weight=2.5, opacity=0.8).add_to(map) #traçage des tournées
display(map)

,Type,NParc,Adresse,Ville,CP,Datedebut,Datefin,idpelle,idloc
0,NaN,A,00Depot,Depot,44119,NaN,NaN,-1,0
2,Mecalac 12,MC10,La Truliere,ST MARS DE COUTAIS,44680,1005.0,10002.0,0,3
3,Mecalac 12,MC10,Rue du Bois Fleurie,LA CHAPELLE SUR ERDRE,44240,1003.0,1005.0,0,5
7,Mecalac 13,MC12,Rue de St Servan,ST HERBLAIN,44800,1005.0,10007.0,1,4
8,Mecalac 13,MC12,Gare Sud,NANTES,44000,1008.0,1005.0,1,2
12,NaN,A,01Depot,Depot,44119,NaN,NaN,-1,1
